In [1]:
import pandas as pd
import numpy as np
# --> 신규설치 라이브러리
import pyproj

# 부산진구 음식점 목록 데이터 가공

In [13]:
### 부산진구 일반음식점 목록 데이터프레임 정의
# - 파일 불러오기
df = pd.read_excel("./data/store_list.xlsx")


# - 동과 사업장명을 합친 컬럼 추가
df["동_사업장명"] = df["소재지전체주소"].str[11:14] + " " + df["사업장명"]

# - 좌표 숫자 데이터 타입으로 변경
df["좌표정보(x)"] = pd.to_numeric(df["좌표정보(x)"], errors="coerce")
df["좌표정보(y)"] = pd.to_numeric(df["좌표정보(y)"], errors="coerce")

df

,사업장명,업태구분명,소재지전화,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),영업상태명,동_사업장명
0,베이비치크,경양식,NaN,부산광역시 부산진구 전포동 309-4,"부산광역시 부산진구 서전로58번길 40, 1층 (전포동)",388180.229380,186050.052148,영업/정상,전포동 베이비치크
1,제이에스푸드,한식,NaN,부산광역시 부산진구 전포동 554-1,"부산광역시 부산진구 전포대로300번길 23, 1층 (전포동)",388178.013427,187030.427351,영업/정상,전포동 제이에스푸드
2,론도테이블,경양식,NaN,부산광역시 부산진구 양정동 273-49,"부산광역시 부산진구 동평로405번길 40, 2층 (양정동)",388562.682440,188197.878283,영업/정상,양정동 론도테이블
3,고향이그리워,한식,NaN,부산광역시 부산진구 부전동 256-6,"부산광역시 부산진구 서면로 56, 1층 타6호, 타15호, 타27호 (부전동)",387468.480136,185990.346087,영업/정상,부전동 고향이그리워
4,김복남맥주 서면아이파크점,호프/통닭,051-907-1717,부산광역시 부산진구 전포동 908 서면아이파크아파트,"부산광역시 부산진구 동성로 50, 상가1동 8호 (전포동, 서면아이파크아파트)",388546.116760,186038.106135,영업/정상,전포동 김복남맥주 서면아이파크점
...,...,...,...,...,...,...,...,...,...
5392,신기한꽃게장개금점,식육(숯불구이),NaN,부산광역시 부산진구 개금동 201-6 번지(1층일부),"부산광역시 부산진구 가야대로 475 (개금동,번지(1층일부))",384362.848527,185689.129625,영업/정상,개금동 신기한꽃게장개금점
5393,소가기가막혀,기타,051-809-0051,부산광역시 부산진구 부전동 516-61,"부산광역시 부산진구 부전로66번길 24-5, 지상1,2층 (부전동)",387282.449271,185989.484647,영업/정상,부전동 소가기가막혀
5394,서면밀면,분식,051-817-2812,부산광역시 부산진구 전포동 667-16번지 이오스프라자 102호,"부산광역시 부산진구 서전로 25 (전포동,이오스프라자 102호)",387863.716122,186257.507510,영업/정상,전포동 서면밀면
5395,가츠모토본점,일식,051-892-8000,부산광역시 부산진구 개금동 1-20,"부산광역시 부산진구 백양관문로77번길 41, 4층 (개금동)",384676.627461,186885.257027,영업/정상,개금동 가츠모토본점


In [14]:
### 좌표 변환 함수 정의
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:2097
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [15]:
# - 좌표계 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# - 바꿀 좌표 데이터
before_coord = np.column_stack((df["좌표정보(x)"], df["좌표정보(y)"]))

# - 좌표 변환 project_array() 함수 실행
after_coord = project_array(before_coord, p1_type, p2_type)

# - 변환된 좌표 df 데이터프레임에 추가
df["경도"] = after_coord[:,0]
df["위도"] = after_coord[:,1]

df

C:\Users\USER\anaconda3\envs\newenv\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\USER\anaconda3\envs\newenv\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\USER\AppData\Local\Temp\ipykernel_17732\1700669751.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-py

,사업장명,업태구분명,소재지전화,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),영업상태명,동_사업장명,경도,위도
0,베이비치크,경양식,NaN,부산광역시 부산진구 전포동 309-4,"부산광역시 부산진구 서전로58번길 40, 1층 (전포동)",388180.229380,186050.052148,영업/정상,전포동 베이비치크,129.063070,35.156031
1,제이에스푸드,한식,NaN,부산광역시 부산진구 전포동 554-1,"부산광역시 부산진구 전포대로300번길 23, 1층 (전포동)",388178.013427,187030.427351,영업/정상,전포동 제이에스푸드,129.063269,35.164862
2,론도테이블,경양식,NaN,부산광역시 부산진구 양정동 273-49,"부산광역시 부산진구 동평로405번길 40, 2층 (양정동)",388562.682440,188197.878283,영업/정상,양정동 론도테이블,129.067755,35.175306
3,고향이그리워,한식,NaN,부산광역시 부산진구 부전동 256-6,"부산광역시 부산진구 서면로 56, 1층 타6호, 타15호, 타27호 (부전동)",387468.480136,185990.346087,영업/정상,부전동 고향이그리워,129.055250,35.155626
4,김복남맥주 서면아이파크점,호프/통닭,051-907-1717,부산광역시 부산진구 전포동 908 서면아이파크아파트,"부산광역시 부산진구 동성로 50, 상가1동 8호 (전포동, 서면아이파크아파트)",388546.116760,186038.106135,영업/정상,전포동 김복남맥주 서면아이파크점,129.067081,35.155855
...,...,...,...,...,...,...,...,...,...,...,...
5392,신기한꽃게장개금점,식육(숯불구이),NaN,부산광역시 부산진구 개금동 201-6 번지(1층일부),"부산광역시 부산진구 가야대로 475 (개금동,번지(1층일부))",384362.848527,185689.129625,영업/정상,개금동 신기한꽃게장개금점,129.021120,35.153486
5393,소가기가막혀,기타,051-809-0051,부산광역시 부산진구 부전동 516-61,"부산광역시 부산진구 부전로66번길 24-5, 지상1,2층 (부전동)",387282.449271,185989.484647,영업/정상,부전동 소가기가막혀,129.053210,35.155653
5394,서면밀면,분식,051-817-2812,부산광역시 부산진구 전포동 667-16번지 이오스프라자 102호,"부산광역시 부산진구 서전로 25 (전포동,이오스프라자 102호)",387863.716122,186257.507510,영업/정상,전포동 서면밀면,129.059646,35.157959
5395,가츠모토본점,일식,051-892-8000,부산광역시 부산진구 개금동 1-20,"부산광역시 부산진구 백양관문로77번길 41, 4층 (개금동)",384676.627461,186885.257027,영업/정상,개금동 가츠모토본점,129.024829,35.164203


In [16]:
### 향후 활용을 고려하여 별도의 파일 생성
df.to_excel("./data/store_list_latlon.xlsx", index=False)

# 기존 리뷰 데이터와 병합

In [17]:
### review_concat 원본 읽어들이기
review_concat = pd.read_csv("./data/review_concat_rating_revised.csv")

### 원본 보존를 위해 카피해 진행...
"""
 [카테고리 필터링]
 - 일반음식점과 상관없는 카테고리 제거
 - "종합분식", "떡볶이"-> "분식"으로 통일 
 - "DDC치킨" -> "치킨,닭강정"으로 변경
 - "브런치카페" -> "브런치"으로 변경
 - 
"""
drop_cat = ["인테리어소품","종합도소매","이벤트,파티","이불,담요","호텔","귀금속,시계","식료품", "수산물가공,제조",
            "스크린야구장","에어컨","패션","영화관","네일아트,네일샵","공방","애플김밥","장례식장"]
df_review = review_concat[review_concat["category"].isin(drop_cat)==False].copy().reset_index(drop=True).iloc[:,1:]

df_review["category"][df_review["category"].isin(["종합분식","떡볶이"])] = "분식"

df_review["category"][df_review["category"].isin(["DDC치킨"])] = "치킨,닭강정"

df_review["category"][df_review["category"].isin(["브런치카페"])] = "브런치"


################[최종사용 변수명:df_review]###################
df_review

C:\Users\USER\AppData\Local\Temp\ipykernel_17732\4205868477.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review["category"][df_review["category"].isin(["종합분식","떡볶이"])] = "분식"
C:\Users\USER\AppData\Local\Temp\ipykernel_17732\4205868477.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review["category"][df_review["category"].isin(["DDC치킨"])] = "치킨,닭강정"
C:\Users\USER\AppData\Local\Temp\ipykernel_17732\4205868477.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

,restaurant_name,review_content,rating,date,visit,register,category
0,전포동 베이비치크,아기자기하고 빈티지한 분위기가 좋았어요ㅎㅎ 명란청양오일파스타랑 트러플버섯크림리조또 ...,NaN,2023-08-02,1,영수증,양식
1,전포동 베이비치크,입구부터 이쁜 공간이 너무 많아서 좋았던 이곳 취향저격 제대로 당해서 안으로 들어왔...,NaN,2023-08-01,1,영수증,양식
2,전포동 베이비치크,와..오랜만에 너무 만족스러운 식사하고 왔어여ㅠ 매장 분위기도 너무 좋고 예쁘신 사...,NaN,2023-07-24,1,영수증,양식
3,전포동 베이비치크,지난번에 너무 맛있게 먹고 이번에 재방문했어요! 베이비치크 플레이트는 진짜 무조건 ...,NaN,2023-08-09,1,영수증,양식
4,전포동 베이비치크,양 진짜 많고 너무 맛있어요ㅠㅠ 안에도 아기자기 앤티크하게 잘 꾸며져있고ㅠㅠㅠ 감성...,NaN,2023-07-05,1,영수증,양식
...,...,...,...,...,...,...,...
114969,양정동 명심이족발,맛있어요,5.0,2020-12-19,1,영수증,"족발,보쌈"
114970,양정동 명심이족발,맛있고 양도 많았어요!,5.0,2020-12-25,2,영수증,"족발,보쌈"
114971,양정동 명심이족발,좋아요,3.5,2020-10-29,1,영수증,"족발,보쌈"
114972,양정동 명심이족발,맛있어요,3.5,2019-12-15,1,영수증,"족발,보쌈"


In [18]:
### cat_concat 원본 읽어들이기
cat_concat = pd.read_csv("./data/cat_concat.csv")

### 원본 보존를 위해 카피해 진행...
# - 위에서 카테고리 필터링한 가게 리스트 생성
rest_list = df_review["restaurant_name"].unique().tolist()
df_cat = cat_concat[cat_concat["restaurant_name"].isin(rest_list)].copy().reset_index(drop=True).iloc[:,1:]


################[최종사용 변수명:df_cat]###################
df_cat

,restaurant_name,ansim,naver_cat_review,naver_cat_count,cat_people
0,전포동 베이비치크,없음,음식이 맛있어요,292,314
1,전포동 베이비치크,없음,인테리어가 멋져요,230,314
2,전포동 베이비치크,없음,친절해요,179,314
3,전포동 베이비치크,없음,특별한 날 가기 좋아요,121,314
4,전포동 베이비치크,없음,특별한 메뉴가 있어요,99,314
...,...,...,...,...,...
18397,가야동 코코샤브가야점,안심식당,특별한 날 가기 좋아요,4,33
18398,가야동 코코샤브가야점,안심식당,양이 많아요,3,33
18399,가야동 코코샤브가야점,안심식당,가성비가 좋아요,2,33
18400,가야동 코코샤브가야점,안심식당,단체모임 하기 좋아요,2,33


### review_concat데이터와 위경도 데이터 합치기

In [19]:
"""
 merge 시 오류가 나서 위와 같이 진행함
 - store_dict 딕셔너리 데이터 가공
   - key : "동 사업장명" 형식의 가게 이름
   - vlaue :"동 사업장명"과 일치하는 사업장명, 업태구분명, 위도, 경도 데이터를 리스트로 받음
   
 - review_loc 각 행 가게이름과 일치하는 데이터를 store_dict의 key 값과 매칭해 value 값을 순서대로 뽑아 리스트로 생성
 - 생성된 리스트로 임시 데이터프레임을 만든 후 본 데이터프레임 review_loc에 병합
 
*** 최종 데이터프레임명 : review_loc
"""

### 카피 데이터프레임에서 작업 진행
review_loc = df_review.copy()


### store_dict 데이터 가공
store_dict = dict(zip(df["동_사업장명"],
                      list(zip(df["사업장명"],df["업태구분명"],df["위도"],df["경도"]))))

# - 위경도 결측치 ['갈매기학교', '가야골추어탕', '단골식당', '꼬김빱', '진영곱창'] 값 별도로 수정
store_dict["부전동 갈매기학교"] = ['갈매기학교', '한식', 35.152439, 129.057434]
store_dict["가야동 가야골추어탕"] = ['가야골추어탕', '한식', 35.148374, 129.030256]
store_dict["당감동 단골식당"] = ['단골식당', '한식', 35.165916, 129.038737]
store_dict["가야동 꼬김빱"] = ['꼬김빱', '한식', 35.1527, 129.0306]
store_dict["범천동 진영곱창"] = ['진영곱창', '한식', 35.144451, 129.059895]


### review_loc 데이터프레임 만들기
# - store_dict값을 review_loc의 행 순서대로 리스트 생성
coord_list = [store_dict[review_loc.iloc[i,0]] for i in range(len(review_loc))]

# - 임시 데이터프레임 생성
temp = pd.DataFrame(coord_list, columns=["사업장명", "업태구분명","위도","경도"])

# - review_loc에 병합
review_loc = pd.concat([review_loc, temp], axis=1)
review_loc["dong"] = review_loc["restaurant_name"].str[:3]


### 열 순서 변경 (+ 위치 수정 가능)
review_loc = review_loc[["restaurant_name", "dong","사업장명","date","visit","category","업태구분명",
                         "위도","경도", "review_content","rating","register"]]


######## 최종 데이터프레임명 : review_loc
review_loc

,restaurant_name,dong,사업장명,date,visit,category,업태구분명,위도,경도,review_content,rating,register
0,전포동 베이비치크,전포동,베이비치크,2023-08-02,1,양식,경양식,35.156031,129.063070,아기자기하고 빈티지한 분위기가 좋았어요ㅎㅎ 명란청양오일파스타랑 트러플버섯크림리조또 ...,NaN,영수증
1,전포동 베이비치크,전포동,베이비치크,2023-08-01,1,양식,경양식,35.156031,129.063070,입구부터 이쁜 공간이 너무 많아서 좋았던 이곳 취향저격 제대로 당해서 안으로 들어왔...,NaN,영수증
2,전포동 베이비치크,전포동,베이비치크,2023-07-24,1,양식,경양식,35.156031,129.063070,와..오랜만에 너무 만족스러운 식사하고 왔어여ㅠ 매장 분위기도 너무 좋고 예쁘신 사...,NaN,영수증
3,전포동 베이비치크,전포동,베이비치크,2023-08-09,1,양식,경양식,35.156031,129.063070,지난번에 너무 맛있게 먹고 이번에 재방문했어요! 베이비치크 플레이트는 진짜 무조건 ...,NaN,영수증
4,전포동 베이비치크,전포동,베이비치크,2023-07-05,1,양식,경양식,35.156031,129.063070,양 진짜 많고 너무 맛있어요ㅠㅠ 안에도 아기자기 앤티크하게 잘 꾸며져있고ㅠㅠㅠ 감성...,NaN,영수증
...,...,...,...,...,...,...,...,...,...,...,...,...
114969,양정동 명심이족발,양정동,명심이족발,2020-12-19,1,"족발,보쌈",식육(숯불구이),35.173053,129.066398,맛있어요,5.0,영수증
114970,양정동 명심이족발,양정동,명심이족발,2020-12-25,2,"족발,보쌈",식육(숯불구이),35.173053,129.066398,맛있고 양도 많았어요!,5.0,영수증
114971,양정동 명심이족발,양정동,명심이족발,2020-10-29,1,"족발,보쌈",식육(숯불구이),35.173053,129.066398,좋아요,3.5,영수증
114972,양정동 명심이족발,양정동,명심이족발,2019-12-15,1,"족발,보쌈",식육(숯불구이),35.173053,129.066398,맛있어요,3.5,영수증


In [20]:
### 파일 저장하기
review_loc.to_excel("./data/review_latlon.xlsx", index=False)

##### 가게별 고윳값 1개만 남기기

In [21]:
review_uniq = review_loc.groupby("restaurant_name").head(1).reset_index(drop=True)

### 고윳값이 아닌 컬럼은 삭제
review_uniq = review_uniq[["restaurant_name", "dong","사업장명","category","업태구분명", "위도","경도"]]

### 파일 저장하기
review_uniq.to_excel("./data/review_latlon_uniq.xlsx", index=False)

### cat_concat데이터와 위경도 데이터 합치기

In [22]:
"""
 merge 시 오류가 나서 위와 같이 진행함
 - store_dict2 딕셔너리 데이터 가공
   - key : "동 사업장명" 형식의 가게 이름
   - vlaue :"동 사업장명"과 일치하는 사업장명, 업태구분명, 위도, 경도 데이터를 리스트로 받음
   
 - cat_loc 각 행 가게이름과 일치하는 데이터를 store_dict2의 key 값과 매칭해 value 값을 순서대로 뽑아 리스트로 생성
 - 생성된 리스트로 임시 데이터프레임을 만든 후 본 데이터프레임 cat_loc에 병합
 
*** 최종 데이터프레임명 : cat_loc
"""

### 카피 데이터프레임에서 작업 진행
cat_loc = df_cat.copy()


### store_dict2 데이터 가공
store_dict2 = dict(zip(df["동_사업장명"],
                      list(zip(df["사업장명"],df["업태구분명"],df["위도"],df["경도"]))))

# - 위경도 결측치 ['갈매기학교', '가야골추어탕', '단골식당', '꼬김빱', '진영곱창'] 값 별도로 수정
store_dict2["부전동 갈매기학교"] = ['갈매기학교', '한식', 35.152439, 129.057434]
store_dict2["가야동 가야골추어탕"] = ['가야골추어탕', '한식', 35.148374, 129.030256]
store_dict2["당감동 단골식당"] = ['단골식당', '한식', 35.165916, 129.038737]
store_dict2["가야동 꼬김빱"] = ['꼬김빱', '한식', 35.1527, 129.0306]
store_dict2["범천동 진영곱창"] = ['진영곱창', '한식', 35.144451, 129.059895]


### cat_loc 데이터프레임 만들기
# - store_dict값을 review_loc의 행 순서대로 리스트 생성
coord_list2 = [store_dict2[cat_loc.iloc[i,0]] for i in range(len(cat_loc))]

# - 임시 데이터프레임 생성
temp2 = pd.DataFrame(coord_list2, columns=["사업장명", "업태구분명","위도","경도"])

# - cat_loc에 병합
cat_loc = pd.concat([cat_loc, temp2], axis=1)
cat_loc["dong"] = cat_loc["restaurant_name"].str[:3]


######## 최종 데이터프레임명 : cat_loc
cat_loc

,restaurant_name,ansim,naver_cat_review,naver_cat_count,cat_people,사업장명,업태구분명,위도,경도,dong
0,전포동 베이비치크,없음,음식이 맛있어요,292,314,베이비치크,경양식,35.156031,129.063070,전포동
1,전포동 베이비치크,없음,인테리어가 멋져요,230,314,베이비치크,경양식,35.156031,129.063070,전포동
2,전포동 베이비치크,없음,친절해요,179,314,베이비치크,경양식,35.156031,129.063070,전포동
3,전포동 베이비치크,없음,특별한 날 가기 좋아요,121,314,베이비치크,경양식,35.156031,129.063070,전포동
4,전포동 베이비치크,없음,특별한 메뉴가 있어요,99,314,베이비치크,경양식,35.156031,129.063070,전포동
...,...,...,...,...,...,...,...,...,...,...
18397,가야동 코코샤브가야점,안심식당,특별한 날 가기 좋아요,4,33,코코샤브가야점,기타,35.155240,129.034172,가야동
18398,가야동 코코샤브가야점,안심식당,양이 많아요,3,33,코코샤브가야점,기타,35.155240,129.034172,가야동
18399,가야동 코코샤브가야점,안심식당,가성비가 좋아요,2,33,코코샤브가야점,기타,35.155240,129.034172,가야동
18400,가야동 코코샤브가야점,안심식당,단체모임 하기 좋아요,2,33,코코샤브가야점,기타,35.155240,129.034172,가야동


In [23]:
### 파일 저장하기
cat_loc.to_excel("./data/cat_latlon.xlsx", index=False)

##### 가게별 고윳값 1개만 남기기

In [24]:
### 리뷰 키워드 한 행에 합치기
# - {가게명:[키워드종합]} 딕셔너리로 정의
kw_dict = dict(cat_loc.groupby("restaurant_name")["naver_cat_review"].unique().items())

# - cat_uniq에 가게명 고윳값과 키워드 종합열 추가
cat_uniq = cat_loc.groupby("restaurant_name").head(1).reset_index(drop=True)
cat_uniq["total_review"] = [kw_dict[cat_uniq.loc[i,"restaurant_name"]] for i in range(len(cat_uniq))]

### 파일 저장하기
cat_uniq.to_excel("./data/cat_latlon_uniq.xlsx", index=False)